In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

# Data

In [3]:
from kegra.utils import load_data, preprocess_adj, get_splits

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 13:31:45.978815 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 13:31:45.986023 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 13:31:45.990471 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import evaluate_preds

In [11]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 13:31:46.000496 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 13:31:46.004836 139757527222080 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 13:31:46.012698 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [12]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 13:31:46.052633 139757527222080 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 13:31:46.119970 139757527222080 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9344 train_acc= 0.3214 val_loss= 1.9347 val_acc= 0.3600 time= 1.0685
Epoch: 0002 train_loss= 1.9216 train_acc= 0.3000 val_loss= 1.9231 val_acc= 0.3500 time= 0.0235
Epoch: 0003 train_loss= 1.9072 train_acc= 0.2929 val_loss= 1.9105 val_acc= 0.3500 time= 0.0223
Epoch: 0004 train_loss= 1.8915 train_acc= 0.2929 val_loss= 1.8966 val_acc= 0.3500 time= 0.0229
Epoch: 0005 train_loss= 1.8746 train_acc= 0.3143 val_loss= 1.8815 val_acc= 0.3533 time= 0.0189
Epoch: 0006 train_loss= 1.8581 train_acc= 0.3000 val_loss= 1.8666 val_acc= 0.3533 time= 0.0182
Epoch: 0007 train_loss= 1.8415 train_acc= 0.2929 val_loss= 1.8518 val_acc= 0.3533 time= 0.0224
Epoch: 0008 train_loss= 1.8248 train_acc= 0.2929 val_loss= 1.8367 val_acc= 0.3500 time= 0.0192
Epoch: 0009 train_loss= 1.8083 train_acc= 0.2929 val_loss= 1.8219 val_acc= 0.3500 time= 0.0184
Epoch: 0010 train_loss= 1.7922 train_acc= 0.2929 val_loss= 1.8077 val_acc= 0.3500 time= 0.0201
Epoch: 0011 train_loss= 1.7766 train_acc= 0.2929 v

Epoch: 0089 train_loss= 0.8906 train_acc= 0.8857 val_loss= 1.1510 val_acc= 0.7500 time= 0.0188
Epoch: 0090 train_loss= 0.8834 train_acc= 0.8857 val_loss= 1.1456 val_acc= 0.7500 time= 0.0211
Epoch: 0091 train_loss= 0.8764 train_acc= 0.8857 val_loss= 1.1407 val_acc= 0.7500 time= 0.0186
Epoch: 0092 train_loss= 0.8697 train_acc= 0.8857 val_loss= 1.1362 val_acc= 0.7500 time= 0.0183
Epoch: 0093 train_loss= 0.8626 train_acc= 0.8857 val_loss= 1.1305 val_acc= 0.7500 time= 0.0186
Epoch: 0094 train_loss= 0.8554 train_acc= 0.8857 val_loss= 1.1243 val_acc= 0.7533 time= 0.0238
Epoch: 0095 train_loss= 0.8485 train_acc= 0.8857 val_loss= 1.1186 val_acc= 0.7533 time= 0.0186
Epoch: 0096 train_loss= 0.8417 train_acc= 0.8857 val_loss= 1.1133 val_acc= 0.7600 time= 0.0189
Epoch: 0097 train_loss= 0.8350 train_acc= 0.8857 val_loss= 1.1076 val_acc= 0.7600 time= 0.0177
Epoch: 0098 train_loss= 0.8286 train_acc= 0.8857 val_loss= 1.1025 val_acc= 0.7667 time= 0.0230
Epoch: 0099 train_loss= 0.8218 train_acc= 0.8857 v

Epoch: 0180 train_loss= 0.5027 train_acc= 0.9500 val_loss= 0.8510 val_acc= 0.7933 time= 0.0201
Epoch: 0181 train_loss= 0.5005 train_acc= 0.9500 val_loss= 0.8483 val_acc= 0.7933 time= 0.0243
Epoch: 0182 train_loss= 0.4986 train_acc= 0.9500 val_loss= 0.8462 val_acc= 0.7967 time= 0.0233
Epoch: 0183 train_loss= 0.4967 train_acc= 0.9500 val_loss= 0.8445 val_acc= 0.7967 time= 0.0189
Epoch: 0184 train_loss= 0.4946 train_acc= 0.9500 val_loss= 0.8422 val_acc= 0.8000 time= 0.0238
Epoch: 0185 train_loss= 0.4916 train_acc= 0.9500 val_loss= 0.8397 val_acc= 0.8000 time= 0.0237
Epoch: 0186 train_loss= 0.4888 train_acc= 0.9500 val_loss= 0.8375 val_acc= 0.8000 time= 0.0201
Epoch: 0187 train_loss= 0.4861 train_acc= 0.9571 val_loss= 0.8355 val_acc= 0.7967 time= 0.0273
Epoch: 0188 train_loss= 0.4836 train_acc= 0.9571 val_loss= 0.8336 val_acc= 0.7867 time= 0.0188
Epoch: 0189 train_loss= 0.4815 train_acc= 0.9571 val_loss= 0.8320 val_acc= 0.7900 time= 0.0187
Epoch: 0190 train_loss= 0.4794 train_acc= 0.9571 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8976
accuracy = 0.8000


# Gaussian Graph Convolution

In [15]:
from gaussian_graph_convolution import GaussianGraphConvolution

In [16]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GaussianGraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9371 train_acc= 0.4429 val_loss= 1.9378 val_acc= 0.3867 time= 0.4000
Epoch: 0002 train_loss= 1.9280 train_acc= 0.4143 val_loss= 1.9294 val_acc= 0.3733 time= 0.0222
Epoch: 0003 train_loss= 1.9183 train_acc= 0.3857 val_loss= 1.9209 val_acc= 0.3700 time= 0.0179
Epoch: 0004 train_loss= 1.9083 train_acc= 0.4286 val_loss= 1.9124 val_acc= 0.3933 time= 0.0180
Epoch: 0005 train_loss= 1.8977 train_acc= 0.4429 val_loss= 1.9036 val_acc= 0.4100 time= 0.0179
Epoch: 0006 train_loss= 1.8869 train_acc= 0.4643 val_loss= 1.8950 val_acc= 0.4433 time= 0.0231
Epoch: 0007 train_loss= 1.8764 train_acc= 0.4714 val_loss= 1.8866 val_acc= 0.4533 time= 0.0223
Epoch: 0008 train_loss= 1.8658 train_acc= 0.4714 val_loss= 1.8781 val_acc= 0.4633 time= 0.0181
Epoch: 0009 train_loss= 1.8550 train_acc= 0.4714 val_loss= 1.8693 val_acc= 0.4633 time= 0.0182
Epoch: 0010 train_loss= 1.8440 train_acc= 0.4714 val_loss= 1.8603 val_acc= 0.4633 time= 0.0227
Epoch: 0011 train_loss= 1.8330 train_acc= 0.4714 v

Epoch: 0096 train_loss= 0.8718 train_acc= 0.8857 val_loss= 1.1429 val_acc= 0.7500 time= 0.0244
Epoch: 0097 train_loss= 0.8641 train_acc= 0.8857 val_loss= 1.1374 val_acc= 0.7500 time= 0.0217
Epoch: 0098 train_loss= 0.8566 train_acc= 0.8857 val_loss= 1.1319 val_acc= 0.7567 time= 0.0194
Epoch: 0099 train_loss= 0.8493 train_acc= 0.8857 val_loss= 1.1262 val_acc= 0.7533 time= 0.0185
Epoch: 0100 train_loss= 0.8423 train_acc= 0.8857 val_loss= 1.1206 val_acc= 0.7533 time= 0.0241
Epoch: 0101 train_loss= 0.8354 train_acc= 0.8857 val_loss= 1.1150 val_acc= 0.7500 time= 0.0187
Epoch: 0102 train_loss= 0.8284 train_acc= 0.8857 val_loss= 1.1089 val_acc= 0.7567 time= 0.0189
Epoch: 0103 train_loss= 0.8213 train_acc= 0.8857 val_loss= 1.1022 val_acc= 0.7533 time= 0.0240
Epoch: 0104 train_loss= 0.8147 train_acc= 0.9000 val_loss= 1.0962 val_acc= 0.7633 time= 0.0198
Epoch: 0105 train_loss= 0.8084 train_acc= 0.9000 val_loss= 1.0906 val_acc= 0.7667 time= 0.0251
Epoch: 0106 train_loss= 0.8023 train_acc= 0.9000 v

Epoch: 0187 train_loss= 0.4934 train_acc= 0.9357 val_loss= 0.8420 val_acc= 0.7900 time= 0.0192
Epoch: 0188 train_loss= 0.4913 train_acc= 0.9429 val_loss= 0.8401 val_acc= 0.7933 time= 0.0193
Epoch: 0189 train_loss= 0.4891 train_acc= 0.9429 val_loss= 0.8383 val_acc= 0.8000 time= 0.0231
Epoch: 0190 train_loss= 0.4868 train_acc= 0.9429 val_loss= 0.8365 val_acc= 0.8000 time= 0.0192
Epoch: 0191 train_loss= 0.4849 train_acc= 0.9429 val_loss= 0.8354 val_acc= 0.8000 time= 0.0184
Epoch: 0192 train_loss= 0.4832 train_acc= 0.9429 val_loss= 0.8351 val_acc= 0.7967 time= 0.0211
Epoch: 0193 train_loss= 0.4820 train_acc= 0.9429 val_loss= 0.8353 val_acc= 0.7933 time= 0.0210
Epoch: 0194 train_loss= 0.4807 train_acc= 0.9429 val_loss= 0.8365 val_acc= 0.7867 time= 0.0221
Epoch: 0195 train_loss= 0.4799 train_acc= 0.9429 val_loss= 0.8380 val_acc= 0.7900 time= 0.0266
Epoch: 0196 train_loss= 0.4785 train_acc= 0.9429 val_loss= 0.8383 val_acc= 0.7867 time= 0.0264
Epoch: 0197 train_loss= 0.4770 train_acc= 0.9429 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8890
accuracy = 0.7770


# Attacks